<a href="https://colab.research.google.com/github/jihyunjeongme/deeplearning-tensflow-3min/blob/master/%5B3%EB%B6%84_%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_5%EC%9E%A5_2_%ED%85%90%EC%84%9C%EB%B3%B4%EB%93%9C%EC%99%80_%EB%AA%A8%EB%8D%B8_%EC%9E%AC%EC%82%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/deeplearning-tensorflow-3mins

/content/gdrive/My Drive/Colab Notebooks/deeplearning-tensorflow-3mins


In [0]:
# 텐서보드는 학습 과정을 추정하는 데 용이
# 학습하는 중간중간 손실값이나 정확도 또는 결로 나온 이미지나 사운드 파일들을
# 다양한 방식으로 시각화해 보여줌

In [0]:
# 먼저 데이터를 읽어 들이는 코드와 플레이스홀더 값들을 똑같이 넣어줌
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


In [0]:

#########
# 신경망 모델 구성
######

# 신경망 모델을 구성할 차례
# 학습 횟수를 카운트 하는 변수(학습에 직접 사용되지는 않음)
# 변수 정의 시 trainable=False라는 옵션을 줌
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 신경망의 각 계층에 다음 코드를 덧붙여 줍니다.
# with tf.name_scope로 묶은 블록은 텐서보드에서 한 계층 내부를 표현
# name='W1' 이름 붙이면 텐서보드에서 해당 이름의 변수가 어디서 사용되는지 쉽게 확인
# 이름은 변수뿐만 아니라 플레이스 홀더, 각각의 연산, 활성화 함수 등 모든 텐서에 붙일 수 있음


# 이렇게 다른 계층들도 전부 tf.name_scope로 묶어주고 이름도 붙여줌
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

    tf.summary.histogram("X", X)
    tf.summary.histogram("Weights", W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

    tf.summary.histogram("Weights", W2)

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

    tf.summary.histogram("Weights", W3)
    tf.summary.histogram("Model", model)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

# 손실값을 추적하기 위해 수집할 값을 지정하는 코드 작성
# tf.summary 모듈의 scalar 함수는 값이 하나인 텐서를 수집할 떄 사용
    tf.summary.scalar('cost', cost)


#########
# 신경망 모델 학습
######


# 모델을 불러들이거나 초기화
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())


# ./model 디렉터리에 기존에 학습해둔 모델이 있는지를 확인해서 
# 모델이 있다면 saver.restore 함수를 사용해 학습된 값들을 불러오고,
# 아니면 변수를 새로 초기화
# 학습된 모델을 저장한 파일을 체크포인트 파일 이라고 함

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# tf.summary.merge_all 함수로 앞서 지정한 텐서들을 수집한 다음
# tf.summary.FileWriter 함수를 이용해 그래프와 텐서들의 값을 저장할 디텍터리 설정
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)


# 최적화를 실행하는 코드를 앞서와 같이 작성
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

#########
# 결과 확인
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Instructions for updating:
Colocations handled automatically by placer.
Step: 1,  Cost: 1.020
Step: 2,  Cost: 0.963
Step: 3,  Cost: 0.915
Step: 4,  Cost: 0.873
Step: 5,  Cost: 0.835
Step: 6,  Cost: 0.803
Step: 7,  Cost: 0.776
Step: 8,  Cost: 0.754
Step: 9,  Cost: 0.735
Step: 10,  Cost: 0.720
Step: 11,  Cost: 0.706
Step: 12,  Cost: 0.694
Step: 13,  Cost: 0.683
Step: 14,  Cost: 0.673
Step: 15,  Cost: 0.663
Step: 16,  Cost: 0.655
Step: 17,  Cost: 0.647
Step: 18,  Cost: 0.640
Step: 19,  Cost: 0.634
Step: 20,  Cost: 0.628
Step: 21,  Cost: 0.623
Step: 22,  Cost: 0.618
Step: 23,  Cost: 0.614
Step: 24,  Cost: 0.609
Step: 25,  Cost: 0.605
Step: 26,  Cost: 0.601
Step: 27,  Cost: 0.596
Step: 28,  Cost: 0.592
Step: 29,  Cost: 0.588
Step: 30,  Cost: 0.585
Step: 31,  Cost: 0.581
Step: 32,  Cost: 0.579
Step: 33,  Cost: 0.576
Step: 34,  Cost: 0.574
Step: 35,  Cost: 0.572
Step: 36,  Cost: 0.570
Step: 37,  Cost: 0.568
Step: 38,  Cost: 0.567
Step: 39,  Cost: 0.565
Step: 40,  Cost: 0.564
Step: 41,  Cost: 